<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/P2_Crear_y_entrenar_CNNs_desde_cero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Práctica 2: Crear y entrenar CNNs desde cero**

Las redes de neuronas convolucionales o simplemente **redes convolucionales** (CNNs, del inglés *convolutional neural networks*), son un tipo de redes neuronales profundas. De hecho son muy similares a estas, dado que también tienen una capa de entrada, una o varias capas ocultas, y una capa de salida,  definidas todas ellas por unos parámetros que se aprenden durante la fase de entrenamiento. Por tanto, una CNN se puede expresar como una función derivable que utiliza los píxeles de una imagen de entrada para obtener unas probabilidades para cada una de las clases objetivo (problema de clasificación) o un valor numérico (problema de regresión).

A continuación, vamos a ver un ejemplo en el que se crea y entrena una CNN desde cero, utilizando la librería [TensorFlow](https://www.tensorflow.org/).

Antes de empezar, vamos a utilizar el método [set_random_seed()](https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed) para establecer el valor de la **semilla** y garantizar la reproducibilidad de los resultados.

In [1]:
from tensorflow.keras.utils import set_random_seed

seed = 121
set_random_seed(seed)  # establece todas las semillas aleatorias del programa (Python, NumPy y TensorFlow)

### **1. Conjunto de datos**

En esta práctica vamos a utilizar un conjunto de datos para clasificación de imágenes denominado [CIFAR10](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10), disponible para descarga en `TensorFlow`. Este conjunto está compuesto por 50.000 imágenes de entrenamiento y 10.000 imágenes de test. Se trata de imágenes en color, de dimensiones espaciales 32x32 y etiquetadas en 10 categorías.

En la web de `TensorFlow` puedes encontrar otros [conjuntos de datos](https://www.tensorflow.org/api_docs/python/tf/keras/datasets).

In [2]:
from tensorflow.keras.datasets import cifar10

# Cargar los datos de CIFAR10 (entrenamiento y test)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


A continuación, vamos a dividir el conjunto de entrenamiento para crear la partición de validación (propociones 80:20). Para ello utilizaremos el método [train_test_split()](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), disponible en la librería `scikit-learn`.

In [3]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de entrenamiento en entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=seed, stratify=y_train)

El siguiente paso consiste en codificar las diferentes clases utilizando el método [`to_categorical()`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical), disponible en `TensorFlow`.

In [4]:
from tensorflow.keras.utils import to_categorical

# Especificar el número de clases y las dimensiones espaciales de las imágenes de entrada
n_classes = 10
img_width = img_height = 32

# Convertir el vector de etiquetas en una matriz binaria para codificar las diferentes clases (one-hot encoding)
y_train = to_categorical(y_train, n_classes)
y_val = to_categorical(y_val, n_classes)
y_test = to_categorical(y_test, n_classes)

Por último, vamos a normalizar los datos de entrada y generar los *batches* necesarios para entrenar la red que se define a continuación. Para ello utilizaremos la clase [`Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset), que permite crear un conjunto de datos a partir de datos de entrada que ya están en memoria.

In [5]:
from tensorflow.data import Dataset

# Crear los conjuntos de datos
train_dataset = Dataset.from_tensor_slices((x_train, y_train))
val_dataset = Dataset.from_tensor_slices((x_val, y_val))
test_dataset = Dataset.from_tensor_slices((x_test, y_test))

print(f'Número de ejemplos del conjunto de entrenamiento: {train_dataset.cardinality().numpy()}')
print(f'Número de ejemplos del conjunto de validación: {val_dataset.cardinality().numpy()}')
print(f'Número de ejemplos del conjunto de test: {test_dataset.cardinality().numpy()}')

# Calcular los valores para la normalización (media y desviación típica)
mean = x_train.mean()
std = x_train.std()

# Función para normalizar las imágenes
def normalize_images(images, labels):
    images = (images - mean) / std
    return images, labels

# Normalizar los datos (normalización global, no por canales)
train_dataset = train_dataset.map(normalize_images)
val_dataset = val_dataset.map(normalize_images)
test_dataset = test_dataset.map(normalize_images)

# Preparar los lotes
batch_size = 256
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

print(f'\nNúmero de lotes del conjunto de entrenamiento: {train_dataset.cardinality().numpy()}')
print(f'Número de lotes del conjunto de validación: {val_dataset.cardinality().numpy()}')
print(f'Número de lotes del conjunto de test: {test_dataset.cardinality().numpy()}')

Número de ejemplos del conjunto de entrenamiento: 40000
Número de ejemplos del conjunto de validación: 10000
Número de ejemplos del conjunto de test: 10000

Número de lotes del conjunto de entrenamiento: 157
Número de lotes del conjunto de validación: 40
Número de lotes del conjunto de test: 40


### **2. Red convolucional**

El siguiente paso consiste en crear una sencilla CNN utilizando las siguientes capas:

*   [Capa convolucional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D): *Conv2D(n_filters, kernel_size)* crea una capa con *n_filters* de tamaño *kernel_size* que se aplican a los datos de entrada para producir un tensor de salidas. Si *use_bias* es `True`, se crea un vector de sesgo y se suma a las salidas. Si *activation* no es `None`, también se aplica la función de activación especificada a las salidas. Otros parámetros relevantes:
> * *strides*: un entero o tupla/lista de dos enteros que especifique el paso de la convolución a lo largo del alto y ancho del volumen de entrada. Especificar un entero implica que se usará el mismo valor para todas las dimensiones espaciales (alto, ancho).
> * *padding*: `valid`, que significa sin relleno; o `same`, que da como resultado un relleno de ceros uniforme (izquieda/derecha y arriba/abajo). Si `padding='same'`y `strides=1`, la salida tiene el mismo tamaño que la entrada.
> * *activation*: función de activación (`relu`, `sigmoid`, etc.) Por defecto, `activation=None` (es decir, no se utiliza función de activación).
> * *input_shape*: cuando se utiliza como primera capa del modelo, es necesario indicar las dimensiones del volumen de entrada; por ejemplo, `input_shape=(128, 128, 3)` para imágenes RGB de 128x128 en formato `data_format="channels_last"`.

*   [Capa max-pooling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D): *MaxPool2D()* reduce los datos de entrada a lo largo de las dimensiones espaciales (alto, ancho) utilizando, para cada canal de la entrada, el valor máximo sobre una ventana de tamaño *pool_size* (por defecto, `pool_size=2`). La ventana se desplaza a lo largo de cada dimensión utilizando el valor del parámetro *strides* (por defecto, `strides=pool_size`).

*   [Capa flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten): *Flatten()* aplana la entrada, convirtiendo un volumen en vector.

*   [Capa completamente conectada](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): *Dense(units)* crea una capa complementamente conectada con el número de neuronas especificado en *units*. Otros parámetros relevantes:
> * *activation*: función de activación (`relu`, `sigmoid`, `softmax`, etc.)
> * *use_bias*: Booleano que indica si la capa utiliza un vector de sesgo (`True`, valor por defecto) o no (`False`).

Por último, vamos a utilizar el método [`summary()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#summary) para imprimir una representación en modo texto de la arquitectura definida. Con este método es posible visualizar también el número de parámetros de cada capa de la red.

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def get_model():

  # Crear un modelo secuencial, compuesto por una secuencia lineal de capas
  model = Sequential()

  # Añadir dos capas convolucionales de 32 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(32, 3, activation='relu', input_shape=(img_width,img_height,3)))  # primera capa (input_shape)
  model.add(Conv2D(32, 3, activation='relu'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  # Añadir dos capas convolucionales de 64 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(64, 3, activation='relu'))
  model.add(Conv2D(64, 3, activation='relu'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  # Transformar el volumen de entrada en un vector
  model.add(Flatten())
  # Añadir una capa completamente conectada 512 neuronas, con ReLU como función de activación
  model.add(Dense(512, activation='relu'))
  # Añadir una última capa completamente conectada con 10 neuronas (número de clases) para obtener la salida de la red, utilizando la función softmax
  model.add(Dense(n_classes, activation='softmax'))

  # Imprimir la representacion en modo texto del modelo
  model.summary()

  return model

### **3. Entrenamiento**

Una vez definida la arquitectura de la CNN, el siguiente paso es configurar el modelo para el entrenamiento. Para ello utilizaremos el método [`compile()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile), siendo estos algunos de sus parámetros más relevantes:
* *optimizer*: nombre del optimizador (`Adam`, `RMSProp`, etc.) y tasa de aprendizaje (`learning_rate`). En la web de `TensorFlow` puedes encontrar otros [optimizadores](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).
* *loss*: función de pérdida (`mean_squared_error`, `binary_crossentropy`, `categorical_crossentropy`, etc.). En la web de `TensorFlow` puedes encontrar otras [funciones de pérdida](https://www.tensorflow.org/api_docs/python/tf/keras/losses).
* *metrics*: métricas que se evalúan para los datos de entrenamiento y validación (`accuracy`, etc.). En la web de `TensorFlow` puedes encontrar otras [métricas](https://www.tensorflow.org/api_docs/python/tf/keras/metrics).


In [15]:
from tensorflow.keras.optimizers import Adam

# Crear el modelo
model = get_model()

# Configurar el proceso de aprendizaje
l_rate = 0.001                    # tasa de aprendizaje
opt = Adam(learning_rate=l_rate)  # optimizador Adam

model.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer=opt,
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_21 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_23 (Conv2D)          (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 8, 8, 64)          0         
 ng2D)                                                

A continuación, vamos a entrenar el modelo para buscar los parámetros que hagan mínima la función de pérdida. Para ello utilizaremos el método [`fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), que necesita que le suministremos los datos de entrenamiento y validación, y el número de *epochs*.

Al finalizar cada *epoch* se mostrará una línea con, por ejemplo, la siguiente información:

> `11s - loss: 1.5593 - acc: 0.4327 - val_loss: 1.2593 - val_acc: 0.5539`

El primer número (`11s`) son los segundos que le ha llevado completar la epoch. `loss` es el valor de la función de pérdida calculado sobre el conjunto de entrenamiento y `val_loss` lo mismo pero calculado sobre el conjunto de validación (cuanto menor, mejor). `acc` y `val_acc` son el ratio de acierto (*accuracy*) calculado sobre el conjunto de entrenamiento y validación, respectivamente (cuanto mayor, mejor).

In [19]:
import numpy as np

# Entrenar el modelo con los datos preparados previamente
history = model.fit(train_dataset,
          epochs=6,   # número de epochs
          verbose=2,  # muestra información al finalizar cada epoch
          validation_data=val_dataset)

# Imprimir el error mínimo de entrenamiento y validación
train_trace = np.array(history.history['loss'])
print(f'\nError mínimo en entrenamiento: {min(train_trace):.6f}')

val_trace = np.array(history.history['val_loss'])
print(f'Error mínimo en validación: {min(val_trace):.6f}')

Epoch 1/6
157/157 - 4s - loss: 0.0371 - accuracy: 0.9872 - val_loss: 3.6652 - val_accuracy: 0.5649 - 4s/epoch - 22ms/step
Epoch 2/6
157/157 - 3s - loss: 0.0360 - accuracy: 0.9885 - val_loss: 3.6124 - val_accuracy: 0.5629 - 3s/epoch - 18ms/step
Epoch 3/6
157/157 - 3s - loss: 0.0350 - accuracy: 0.9878 - val_loss: 3.7093 - val_accuracy: 0.5631 - 3s/epoch - 19ms/step
Epoch 4/6
157/157 - 3s - loss: 0.0356 - accuracy: 0.9883 - val_loss: 3.7897 - val_accuracy: 0.5607 - 3s/epoch - 21ms/step
Epoch 5/6
157/157 - 3s - loss: 0.0300 - accuracy: 0.9901 - val_loss: 3.8502 - val_accuracy: 0.5555 - 3s/epoch - 19ms/step
Epoch 6/6
157/157 - 3s - loss: 0.0276 - accuracy: 0.9911 - val_loss: 4.0291 - val_accuracy: 0.5542 - 3s/epoch - 18ms/step

Error mínimo en entrenamiento: 0.027575
Error mínimo en validación: 3.612392


### **4. Evaluación**

Hemos visto cómo crear y entrenar una CNN desde cero, utilizando una configuración de hiperparámetros que no necesariamente es la mejor. Lo ideal sería realizar una búsqueda de hiperparámetros y, una vez obtenida la mejor configuración, evaluar el modelo sobre el conjunto de test y así obtener el resultado final.

A continuación, se muestra el código para evaluar el modelo final en el conjunto de test utilizando el método [`evaluate()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate).

In [17]:
# Evaluar el modelo en el conjunto de test
test_loss, test_acc = model.evaluate(test_dataset, verbose=1)
print(f'test_loss: {test_loss:.4f}, test_acc: {test_acc:.4f}')

40/40 [==============================] - 0s 7ms/step - loss: 3.4793 - accuracy: 0.5719
test_loss: 3.4793, test_acc: 0.5719


Por último, además de analizar el error obtenido, podemos hacer predicciones con el modelo entrenado. Para ello utilizaremos el método [`predict()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict), al que le suministraremos los datos sobre los que realizar las predicciones (en este caso, los datos de test).

In [18]:
# Obtener las predicciones para todos los ejemplos del conjunto de test
predictions = model.predict(test_dataset, verbose=1)

# Imprimir la predicción para, por ejemplo, las cinco primeras imágenes
for i in range(0,5):
  print(f'Predicción imagen {i} - Clase: {np.argmax([predictions[i]])}, Probabilidad: {np.max(predictions[i]):.4f}')

40/40 [==============================] - 0s 7ms/step
Predicción imagen 0 - Clase: 3, Probabilidad: 1.0000
Predicción imagen 1 - Clase: 8, Probabilidad: 0.9924
Predicción imagen 2 - Clase: 8, Probabilidad: 0.7032
Predicción imagen 3 - Clase: 0, Probabilidad: 0.9997
Predicción imagen 4 - Clase: 4, Probabilidad: 0.3834


### **5. Ejercicios**



**EJERCICIO 1**

Modifica las capas convolucionales del modelo original utilizando el parámetro `padding='same'`.

¿Cómo afecta este cambio en la arquitectura? Analiza las dimensiones de salida y el número de parámetros de cada capa.

In [12]:
def get_model2():

  # Crear un modelo secuencial, compuesto por una secuencia lineal de capas
  model = Sequential()

  # Añadir dos capas convolucionales de 32 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(32, 3, activation='relu', input_shape=(img_width,img_height,3), padding = 'same'))  # primera capa (input_shape)
  model.add(Conv2D(32, 3, activation='relu', padding = 'same'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  # Añadir dos capas convolucionales de 64 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(64, 3, activation='relu', padding = 'same'))
  model.add(Conv2D(64, 3, activation='relu', padding = 'same'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  # Transformar el volumen de entrada en un vector
  model.add(Flatten())
  # Añadir una capa completamente conectada 512 neuronas, con ReLU como función de activación
  model.add(Dense(512, activation='relu'))
  # Añadir una última capa completamente conectada con 10 neuronas (número de clases) para obtener la salida de la red, utilizando la función softmax
  model.add(Dense(n_classes, activation='softmax'))

  # Imprimir la representacion en modo texto del modelo
  model.summary()

  return model
# Crear el modelo
model2 = get_model2()

# Configurar el proceso de aprendizaje
l_rate = 0.001                    # tasa de aprendizaje
opt = Adam(learning_rate=l_rate)  # optimizador Adam

model2.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer=opt,
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                 

**EJERCICIO 2**

Partiendo del modelo obtenido en el ejercicio 1, añade un nuevo bloque de capas convolucionales antes de la capa *flatten* con las siguientes indicaciones: dos capas convolucionales de 128 filtros cada una y dimensiones espaciales 3x3, sin utilizar *padding*. Estas dos capas deberán ir seguidas de una capa *max-pooling* con tamaño de ventana 2.

¿Qué impacto tiene en el número de parámetros total añadir estas tres capas? ¿Sería posible hacer esta modificación en el modelo original?

In [13]:
def get_model3():

  # Crear un modelo secuencial, compuesto por una secuencia lineal de capas
  model = Sequential()

  # Añadir dos capas convolucionales de 32 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(32, 3, activation='relu', input_shape=(img_width,img_height,3), padding = 'same'))  # primera capa (input_shape)
  model.add(Conv2D(32, 3, activation='relu', padding = 'same'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  # Añadir dos capas convolucionales de 64 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(64, 3, activation='relu', padding = 'same'))
  model.add(Conv2D(64, 3, activation='relu', padding = 'same'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  model.add(Conv2D(128, 3, activation='relu'))
  model.add(Conv2D(128, 3, activation='relu'))

  model.add(MaxPooling2D())

  # Transformar el volumen de entrada en un vector
  model.add(Flatten())
  # Añadir una capa completamente conectada 512 neuronas, con ReLU como función de activación
  model.add(Dense(512, activation='relu'))
  # Añadir una última capa completamente conectada con 10 neuronas (número de clases) para obtener la salida de la red, utilizando la función softmax
  model.add(Dense(n_classes, activation='softmax'))

  # Imprimir la representacion en modo texto del modelo
  model.summary()

  return model
# Crear el modelo
model3 = get_model3()

# Configurar el proceso de aprendizaje
l_rate = 0.001                    # tasa de aprendizaje
opt = Adam(learning_rate=l_rate)  # optimizador Adam

model3.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer=opt,
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_9 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                 

Es mejor porque tiene menos parámetros al añadir otra capa despues del pooling, es por esto que aumentamos la profundidad disminuyendo el número de parámetros a la vez.

In [14]:
def get_model4():

  # Crear un modelo secuencial, compuesto por una secuencia lineal de capas
  model = Sequential()

  # Añadir dos capas convolucionales de 32 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(32, 3, activation='relu', input_shape=(img_width,img_height,3)))  # primera capa (input_shape)
  model.add(Conv2D(32, 3, activation='relu'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  # Añadir dos capas convolucionales de 64 filtros (dimensiones 3x3), con ReLU como función de activación
  model.add(Conv2D(64, 3, activation='relu'))
  model.add(Conv2D(64, 3, activation='relu'))
  # Añadir una capa max-pooling con tamaño de ventana 2
  model.add(MaxPooling2D())

  model.add(Conv2D(128, 3, activation='relu'))
  model.add(Conv2D(128, 3, activation='relu'))

  model.add(MaxPooling2D())
  # Transformar el volumen de entrada en un vector
  model.add(Flatten())
  # Añadir una capa completamente conectada 512 neuronas, con ReLU como función de activación
  model.add(Dense(512, activation='relu'))
  # Añadir una última capa completamente conectada con 10 neuronas (número de clases) para obtener la salida de la red, utilizando la función softmax
  model.add(Dense(n_classes, activation='softmax'))

  # Imprimir la representacion en modo texto del modelo
  model.summary()

  return model

# Crear el modelo
model4 = get_model4()

# Configurar el proceso de aprendizaje
l_rate = 0.001                    # tasa de aprendizaje
opt = Adam(learning_rate=l_rate)  # optimizador Adam

model4.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer=opt,
              metrics=['accuracy'])

ValueError: Exception encountered when calling layer "max_pooling2d_9" (type MaxPooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_9/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,1,128].

Call arguments received by layer "max_pooling2d_9" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 1, 1, 128), dtype=float32)

El número de parámetros es muy pequeño.